# Closures in Python

---

## Table of Contents
1. What is a Closure?
2. Nested Functions
3. Free Variables and Cell Objects
4. Creating Closures
5. The nonlocal Keyword
6. Closure vs Class
7. Practical Use Cases
8. Common Pitfalls
9. Key Points
10. Practice Exercises

---

## 1. What is a Closure?

A **closure** is a function object that remembers values in the enclosing scope even when that scope has finished executing.

**Three requirements for a closure:**
1. There must be a nested function (function inside a function)
2. The nested function must refer to variables defined in the enclosing function
3. The enclosing function must return the nested function

In [2]:
# Simple closure example
def outer_function(message):
    # 'message' is a free variable for inner_function
    
    def inner_function():
        # inner_function "closes over" message
        print(message)
    
    return inner_function  # Return the function, don't call it

# Create closures
hello_func = outer_function("Hello!")
goodbye_func = outer_function("Goodbye!")

# outer_function has finished, but the closures remember 'message'
hello_func()    # Prints: Hello!
goodbye_func()  # Prints: Goodbye!

Hello!
Goodbye!


In [3]:
# Visualizing the closure
def make_multiplier(n):
    def multiplier(x):
        return x * n
    return multiplier

times_3 = make_multiplier(3)
times_5 = make_multiplier(5)

print(f"times_3(10) = {times_3(10)}")
print(f"times_5(10) = {times_5(10)}")

# Check if it's a closure
print(f"\nIs times_3 a closure? {times_3.__closure__ is not None}")
print(f"Free variables: {times_3.__code__.co_freevars}")
print(f"Closure contents: {times_3.__closure__[0].cell_contents}")

times_3(10) = 30
times_5(10) = 50

Is times_3 a closure? True
Free variables: ('n',)
Closure contents: 3


---

## 2. Nested Functions

Understanding nested functions is essential for closures.

In [4]:
# Basic nested function (not a closure - no free variables returned)
def outer():
    x = 10
    
    def inner():
        print(f"x from outer: {x}")
    
    inner()  # Called inside outer, not returned

outer()

x from outer: 10


In [5]:
# Multiple levels of nesting
def level1(a):
    def level2(b):
        def level3(c):
            return a + b + c
        return level3
    return level2

# Build up the closure chain
add_10 = level1(10)
add_10_20 = add_10(20)
result = add_10_20(30)

print(f"Result: {result}")  # 10 + 20 + 30 = 60

# Or chain the calls
print(f"Chained: {level1(1)(2)(3)}")  # 1 + 2 + 3 = 6

Result: 60
Chained: 6


In [6]:
# Scope lookup: LEGB rule
# L: Local, E: Enclosing, G: Global, B: Built-in

x = "global"

def outer():
    x = "enclosing"
    
    def inner():
        x = "local"
        print(f"Inner: {x}")  # Local
    
    inner()
    print(f"Outer: {x}")  # Enclosing

outer()
print(f"Global: {x}")  # Global

Inner: local
Outer: enclosing
Global: global


---

## 3. Free Variables and Cell Objects

In [7]:
# Free variables are stored in __closure__
def make_counter(start=0):
    count = start
    
    def counter():
        nonlocal count
        count += 1
        return count
    
    return counter

counter = make_counter(10)

# Inspect the closure
print(f"Function name: {counter.__name__}")
print(f"Free variables: {counter.__code__.co_freevars}")
print(f"Closure: {counter.__closure__}")

# Access cell contents
for i, cell in enumerate(counter.__closure__):
    print(f"Cell {i}: {cell.cell_contents}")

Function name: counter
Free variables: ('count',)
Closure: (<cell at 0x000002CBF48C7F40: int object at 0x00007FFD107B44C8>,)
Cell 0: 10


In [8]:
# Multiple free variables
def create_formatter(prefix, suffix):
    def format_text(text):
        return f"{prefix}{text}{suffix}"
    return format_text

html_bold = create_formatter("<b>", "</b>")
html_italic = create_formatter("<i>", "</i>")

print(html_bold("Hello"))
print(html_italic("World"))

# Check closure contents
print(f"\nFree vars: {html_bold.__code__.co_freevars}")
for var, cell in zip(html_bold.__code__.co_freevars, html_bold.__closure__):
    print(f"  {var} = {cell.cell_contents!r}")

<b>Hello</b>
<i>World</i>

Free vars: ('prefix', 'suffix')
  prefix = '<b>'
  suffix = '</b>'


---

## 4. Creating Closures

In [9]:
# Closure as a function factory
def power_factory(exponent):
    """Create a function that raises numbers to a power."""
    def power(base):
        return base ** exponent
    return power

square = power_factory(2)
cube = power_factory(3)
fourth = power_factory(4)

print(f"square(5) = {square(5)}")
print(f"cube(5) = {cube(5)}")
print(f"fourth(5) = {fourth(5)}")

square(5) = 25
cube(5) = 125
fourth(5) = 625


In [10]:
# Closure with mutable state
def make_averager():
    """Create a running average calculator."""
    series = []  # Mutable object
    
    def averager(new_value):
        series.append(new_value)
        return sum(series) / len(series)
    
    return averager

avg = make_averager()
print(f"avg(10) = {avg(10)}")
print(f"avg(20) = {avg(20)}")
print(f"avg(30) = {avg(30)}")
print(f"avg(40) = {avg(40)}")

avg(10) = 10.0
avg(20) = 15.0
avg(30) = 20.0
avg(40) = 25.0


In [11]:
# More efficient averager using nonlocal
def make_efficient_averager():
    """Running average without storing all values."""
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_efficient_averager()
print(f"avg(10) = {avg(10)}")
print(f"avg(20) = {avg(20)}")
print(f"avg(30) = {avg(30)}")

avg(10) = 10.0
avg(20) = 15.0
avg(30) = 20.0


---

## 5. The nonlocal Keyword

`nonlocal` allows modification of variables in the enclosing scope.

In [12]:
# Without nonlocal - creates new local variable
def outer_no_nonlocal():
    count = 0
    
    def inner():
        count = 1  # Creates new local 'count', doesn't modify outer
        return count
    
    inner()
    return count  # Still 0

print(f"Without nonlocal: {outer_no_nonlocal()}")

Without nonlocal: 0


In [13]:
# With nonlocal - modifies enclosing variable
def outer_with_nonlocal():
    count = 0
    
    def inner():
        nonlocal count
        count = 1  # Modifies outer's count
        return count
    
    inner()
    return count  # Now 1

print(f"With nonlocal: {outer_with_nonlocal()}")

With nonlocal: 1


In [14]:
# Practical example: counter with reset
def make_counter_with_reset(start=0):
    count = start
    
    def counter():
        nonlocal count
        count += 1
        return count
    
    def reset():
        nonlocal count
        count = start
    
    def get_count():
        return count
    
    # Return multiple functions sharing the same closure
    counter.reset = reset
    counter.get = get_count
    return counter

c = make_counter_with_reset()
print(f"Count: {c()}, {c()}, {c()}")
print(f"Current: {c.get()}")
c.reset()
print(f"After reset: {c()}, {c()}")

Count: 1, 2, 3
Current: 3
After reset: 1, 2


In [15]:
# nonlocal vs global
x = "global"

def outer():
    x = "enclosing"
    
    def inner_nonlocal():
        nonlocal x  # Refers to enclosing x
        x = "modified by nonlocal"
    
    def inner_global():
        global x  # Refers to global x
        x = "modified by global"
    
    print(f"Before: enclosing x = {x}")
    inner_nonlocal()
    print(f"After nonlocal: enclosing x = {x}")

print(f"Global x before: {x}")
outer()
print(f"Global x after: {x}")

Global x before: global
Before: enclosing x = enclosing
After nonlocal: enclosing x = modified by nonlocal
Global x after: global


---

## 6. Closure vs Class

Closures can often replace simple classes.

In [16]:
# Class-based approach
class Counter:
    def __init__(self, start=0):
        self.count = start
    
    def __call__(self):
        self.count += 1
        return self.count

class_counter = Counter()
print(f"Class: {class_counter()}, {class_counter()}, {class_counter()}")

Class: 1, 2, 3


In [17]:
# Closure-based approach
def make_counter(start=0):
    count = start
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

closure_counter = make_counter()
print(f"Closure: {closure_counter()}, {closure_counter()}, {closure_counter()}")

Closure: 1, 2, 3


In [18]:
# Comparison
import sys

# Memory comparison
print(f"Class instance size: {sys.getsizeof(class_counter)} + dict: {sys.getsizeof(class_counter.__dict__)}")
print(f"Closure size: {sys.getsizeof(closure_counter)} + closure: {sys.getsizeof(closure_counter.__closure__)}")

Class instance size: 48 + dict: 296
Closure size: 160 + closure: 48


In [19]:
# When to use classes vs closures

# Use CLOSURE when:
# - Simple state with few operations
# - Function factory pattern
# - Callback functions
# - Lightweight and functional style

# Use CLASS when:
# - Complex state with many operations
# - Need inheritance
# - Need multiple methods
# - State needs to be inspected/modified externally

# Complex example better as class
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.transactions = []
    
    def deposit(self, amount):
        self.balance += amount
        self.transactions.append(('deposit', amount))
    
    def withdraw(self, amount):
        if amount <= self.balance:
            self.balance -= amount
            self.transactions.append(('withdraw', amount))
            return True
        return False
    
    def get_history(self):
        return self.transactions.copy()

---

## 7. Practical Use Cases

In [20]:
# 1. Configuration-based function creation
def make_logger(level, prefix):
    """Create a logger function with preset configuration."""
    def logger(message):
        print(f"[{level}] {prefix}: {message}")
    return logger

debug_log = make_logger("DEBUG", "App")
error_log = make_logger("ERROR", "App")
db_log = make_logger("INFO", "Database")

debug_log("Starting application")
error_log("Connection failed")
db_log("Query executed")

[DEBUG] App: Starting application
[ERROR] App: Connection failed
[INFO] Database: Query executed


In [21]:
# 2. Memoization (caching)
def memoize(func):
    """Closure-based memoization decorator."""
    cache = {}
    
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    
    wrapper.cache = cache  # Expose cache for inspection
    return wrapper

@memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fib(30) = {fibonacci(30)}")
print(f"Cache size: {len(fibonacci.cache)}")

fib(30) = 832040
Cache size: 31


In [22]:
# 3. Event handlers with context
def make_button_handler(button_name, action):
    """Create button click handler with context."""
    click_count = 0
    
    def handler():
        nonlocal click_count
        click_count += 1
        print(f"Button '{button_name}' clicked {click_count} times")
        action()
    
    return handler

def save_action():
    print("  -> Saving document...")

def cancel_action():
    print("  -> Cancelling operation...")

save_handler = make_button_handler("Save", save_action)
cancel_handler = make_button_handler("Cancel", cancel_action)

# Simulate button clicks
save_handler()
save_handler()
cancel_handler()

Button 'Save' clicked 1 times
  -> Saving document...
Button 'Save' clicked 2 times
  -> Saving document...
Button 'Cancel' clicked 1 times
  -> Cancelling operation...


In [23]:
# 4. Partial function application
def partial(func, *partial_args, **partial_kwargs):
    """Create partial function (like functools.partial)."""
    def wrapper(*args, **kwargs):
        all_kwargs = {**partial_kwargs, **kwargs}
        return func(*partial_args, *args, **all_kwargs)
    return wrapper

def greet(greeting, name, punctuation="!"):
    return f"{greeting}, {name}{punctuation}"

say_hello = partial(greet, "Hello")
say_hi_excited = partial(greet, "Hi", punctuation="!!!")

print(say_hello("Alice"))
print(say_hello("Bob", punctuation="."))
print(say_hi_excited("Charlie"))

Hello, Alice!
Hello, Bob.
Hi, Charlie!!!


In [24]:
# 5. Rate limiter
import time

def rate_limiter(max_calls, period):
    """Limit function calls to max_calls per period seconds."""
    calls = []
    
    def limiter(func):
        def wrapper(*args, **kwargs):
            nonlocal calls
            now = time.time()
            # Remove old calls outside the period
            calls = [t for t in calls if now - t < period]
            
            if len(calls) >= max_calls:
                wait_time = period - (now - calls[0])
                print(f"Rate limited! Wait {wait_time:.1f}s")
                return None
            
            calls.append(now)
            return func(*args, **kwargs)
        return wrapper
    return limiter

@rate_limiter(max_calls=3, period=5)
def api_call(endpoint):
    return f"Called {endpoint}"

for i in range(5):
    result = api_call(f"/api/resource/{i}")
    print(f"Call {i}: {result}")

Call 0: Called /api/resource/0
Call 1: Called /api/resource/1
Call 2: Called /api/resource/2
Rate limited! Wait 5.0s
Call 3: None
Rate limited! Wait 5.0s
Call 4: None


---

## 8. Common Pitfalls

In [25]:
# Pitfall 1: Late binding in loops
# WRONG - all functions capture the same variable
def create_multipliers_wrong():
    multipliers = []
    for i in range(5):
        def multiplier(x):
            return x * i  # 'i' is looked up at call time, not definition time
        multipliers.append(multiplier)
    return multipliers

wrong_multipliers = create_multipliers_wrong()
print("Wrong (all use i=4):")
for m in wrong_multipliers:
    print(f"  {m(10)}")

Wrong (all use i=4):
  40
  40
  40
  40
  40


In [26]:
# Solution 1: Default argument (captures value at definition time)
def create_multipliers_fixed1():
    multipliers = []
    for i in range(5):
        def multiplier(x, i=i):  # Default argument captures current value
            return x * i
        multipliers.append(multiplier)
    return multipliers

fixed1 = create_multipliers_fixed1()
print("Fixed with default arg:")
for idx, m in enumerate(fixed1):
    print(f"  multiplier {idx}: {m(10)}")

Fixed with default arg:
  multiplier 0: 0
  multiplier 1: 10
  multiplier 2: 20
  multiplier 3: 30
  multiplier 4: 40


In [27]:
# Solution 2: Factory function
def create_multipliers_fixed2():
    def make_multiplier(n):
        def multiplier(x):
            return x * n
        return multiplier
    
    return [make_multiplier(i) for i in range(5)]

fixed2 = create_multipliers_fixed2()
print("Fixed with factory:")
for idx, m in enumerate(fixed2):
    print(f"  multiplier {idx}: {m(10)}")

Fixed with factory:
  multiplier 0: 0
  multiplier 1: 10
  multiplier 2: 20
  multiplier 3: 30
  multiplier 4: 40


In [28]:
# Pitfall 2: Modifying closure variables without nonlocal
def broken_counter():
    count = 0
    def increment():
        # count += 1  # UnboundLocalError!
        # Python sees 'count' on left side of assignment and treats it as local
        pass
    return increment

# Solution: use nonlocal
def working_counter():
    count = 0
    def increment():
        nonlocal count
        count += 1
        return count
    return increment

counter = working_counter()
print(f"Counter: {counter()}, {counter()}, {counter()}")

Counter: 1, 2, 3


In [29]:
# Pitfall 3: Mutable default arguments (not specific to closures but related)
def append_to_wrong(item, target=[]):  # Don't do this!
    target.append(item)
    return target

print(append_to_wrong(1))  # [1]
print(append_to_wrong(2))  # [1, 2] - oops!

def append_to_correct(item, target=None):
    if target is None:
        target = []
    target.append(item)
    return target

print(append_to_correct(1))  # [1]
print(append_to_correct(2))  # [2] - correct!

[1]
[1, 2]
[1]
[2]


---

## 9. Key Points

1. **Closure Definition**: Function + its enclosing scope's free variables
2. **Requirements**: Nested function, free variables, returned function
3. **Free Variables**: Variables from enclosing scope used by nested function
4. **nonlocal**: Required to modify (not just read) enclosing variables
5. **Cell Objects**: Store free variables in `__closure__`
6. **Late Binding**: Variables looked up at call time, not definition time
7. **Use Cases**: Function factories, decorators, callbacks, memoization
8. **vs Classes**: Closures are lighter, classes are more flexible
9. **Loop Pitfall**: Use default args or factories to capture loop variables
10. **Inspection**: Use `__closure__` and `__code__.co_freevars`

---

## 10. Practice Exercises

In [30]:
# Exercise 1: Create a make_accumulator function
# - Returns a function that adds to a running total
# - Should support optional initial value
# - Include get_total() and reset() attached functions

def make_accumulator(initial=0):
    pass

# Test:
# acc = make_accumulator(100)
# print(acc(10))  # 110
# print(acc(20))  # 130
# print(acc.get_total())  # 130
# acc.reset()
# print(acc.get_total())  # 100

In [31]:
# Exercise 2: Create a once function
# - Returns a function that can only be called once
# - Subsequent calls return the cached result
# - Include was_called() to check if called

def once(func):
    pass

# Test:
# @once
# def initialize():
#     print("Initializing...")
#     return "Initialized"
# print(initialize())  # Initializing... Initialized
# print(initialize())  # Initialized (no print)
# print(initialize.was_called())  # True

In [32]:
# Exercise 3: Create a compose function
# - Takes multiple functions as arguments
# - Returns a function that applies them right to left
# - compose(f, g, h)(x) == f(g(h(x)))

def compose(*functions):
    pass

# Test:
# double = lambda x: x * 2
# add_one = lambda x: x + 1
# square = lambda x: x ** 2
# f = compose(double, add_one, square)  # double(add_one(square(x)))
# print(f(3))  # double(add_one(9)) = double(10) = 20

In [33]:
# Exercise 4: Create a debounce function
# - Delays function execution until wait_time has passed
# - If called again before wait_time, resets the timer
# - For simplicity, just track timestamps (no actual delay)

import time

def debounce(wait_time):
    pass

# Test:
# @debounce(0.5)
# def save_data(data):
#     print(f"Saved: {data}")
# save_data("v1")  # Debounced
# time.sleep(0.6)
# save_data("v2")  # Saved: v2

In [34]:
# Exercise 5: Create a curry function
# - Converts a function to curried form
# - curry(f)(a)(b)(c) == f(a, b, c)
# - Should work with any number of arguments

def curry(func):
    pass

# Test:
# def add3(a, b, c):
#     return a + b + c
# curried_add = curry(add3)
# print(curried_add(1)(2)(3))  # 6
# print(curried_add(1, 2)(3))  # 6
# print(curried_add(1)(2, 3))  # 6

---

## Solutions

In [35]:
# Solution 1:
def make_accumulator(initial=0):
    total = initial
    start = initial
    
    def accumulate(value):
        nonlocal total
        total += value
        return total
    
    def get_total():
        return total
    
    def reset():
        nonlocal total
        total = start
    
    accumulate.get_total = get_total
    accumulate.reset = reset
    return accumulate

acc = make_accumulator(100)
print(f"acc(10) = {acc(10)}")
print(f"acc(20) = {acc(20)}")
print(f"get_total() = {acc.get_total()}")
acc.reset()
print(f"After reset: {acc.get_total()}")

acc(10) = 110
acc(20) = 130
get_total() = 130
After reset: 100


In [36]:
# Solution 2:
from functools import wraps

def once(func):
    called = False
    result = None
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal called, result
        if not called:
            result = func(*args, **kwargs)
            called = True
        return result
    
    def was_called():
        return called
    
    wrapper.was_called = was_called
    return wrapper

@once
def initialize():
    print("Initializing...")
    return "Initialized"

print(f"First call: {initialize()}")
print(f"Second call: {initialize()}")
print(f"Was called: {initialize.was_called()}")

Initializing...
First call: Initialized
Second call: Initialized
Was called: True


In [37]:
# Solution 3:
def compose(*functions):
    if not functions:
        return lambda x: x
    
    def composed(x):
        result = x
        for func in reversed(functions):
            result = func(result)
        return result
    
    return composed

double = lambda x: x * 2
add_one = lambda x: x + 1
square = lambda x: x ** 2

f = compose(double, add_one, square)
print(f"compose(double, add_one, square)(3) = {f(3)}")
# square(3) = 9, add_one(9) = 10, double(10) = 20

# Verify step by step
print(f"Step by step: double(add_one(square(3))) = {double(add_one(square(3)))}")

compose(double, add_one, square)(3) = 20
Step by step: double(add_one(square(3))) = 20


In [38]:
# Solution 4:
import time
from functools import wraps

def debounce(wait_time):
    def decorator(func):
        last_call_time = [0]
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()
            time_since_last = current_time - last_call_time[0]
            
            if time_since_last < wait_time:
                print(f"Debounced (wait {wait_time - time_since_last:.2f}s more)")
                return None
            
            last_call_time[0] = current_time
            return func(*args, **kwargs)
        
        return wrapper
    return decorator

@debounce(0.5)
def save_data(data):
    print(f"Saved: {data}")
    return True

save_data("v1")  # Saved
save_data("v2")  # Debounced
time.sleep(0.6)
save_data("v3")  # Saved

Saved: v1
Debounced (wait 0.50s more)
Saved: v3


True

In [39]:
# Solution 5:
import inspect

def curry(func):
    num_args = len(inspect.signature(func).parameters)
    
    def curried(*args):
        if len(args) >= num_args:
            return func(*args[:num_args])
        
        def more(*more_args):
            return curried(*(args + more_args))
        
        return more
    
    return curried

def add3(a, b, c):
    return a + b + c

curried_add = curry(add3)
print(f"curried_add(1)(2)(3) = {curried_add(1)(2)(3)}")
print(f"curried_add(1, 2)(3) = {curried_add(1, 2)(3)}")
print(f"curried_add(1)(2, 3) = {curried_add(1)(2, 3)}")
print(f"curried_add(1, 2, 3) = {curried_add(1, 2, 3)}")

curried_add(1)(2)(3) = 6
curried_add(1, 2)(3) = 6
curried_add(1)(2, 3) = 6
curried_add(1, 2, 3) = 6
